In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

# from google.colab import drive
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
from tqdm import tqdm
import pandas as pd
# import nltk

# !pip install parlai
from parlai.core.agents import create_agent_from_model_file
from parlai.core.teachers import register_teacher, DialogTeacher
from parlai.scripts.eval_model import EvalModel
from parlai.utils.safety import OffensiveStringMatcher, OffensiveLanguageClassifier

Defaulting to user installation because normal site-packages is not writeable
  Using cached parlai-1.6.0-py3-none-any.whl (1.7 MB)
  Using cached myst_parser-0.12.10-py3-none-any.whl (34 kB)
  Using cached Sphinx-2.2.2-py3-none-any.whl (2.7 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached attrs-20.2.0-py2.py3-none-any.whl (48 kB)
     |████████████████████████████████| 107 kB 39.7 MB/s eta 0:00:01
  Using cached sphinx_rtd_theme-1.0.0-py2.py3-none-any.whl (2.8 MB)
  Using cached torchtext-0.12.0-cp39-cp39-manylinux1_x86_64.whl (10.4 MB)
  Using cached flake8-4.0.1-py2.py3-none-any.whl (64 kB)
  Using cached pytest_regressions-2.3.1-py3-none-any.whl (22 kB)
  Using cached flake8_bugbear-22.3.23-py3-none-any.whl (19 kB)
     |████████████████████████████████| 147 kB 63.7 MB/s eta 0:00:01
  Using cached emoji-1.7.0-py3-none-any.whl
  Using cached requests_mock-1.9.3-py2.py3-none-any.whl (27 kB)
  Using cached sphinx_autodoc_typehints-1.10.3-py3-none-any.whl (8.4 kB)
 

# GPT2 with Fine Tuning

### Prepare data

In [ ]:
@register_teacher("test_cases")
class MyTeacher(DialogTeacher):
  def __init__(self, opt, shared=None):
    opt['datafile'] = f'./data/query/test_cases.txt'
    super().__init__(opt, shared)
  
  def setup_data(self, datafile):
    print(f" ~~ Loading from {datafile} ~~ ")
    with open(self.opt['datafile']) as f:
      lines = [line.strip() for line in f]

    # Get first dialogue utterances written by humans
    for text in lines:
      yield (text, '__notok__'), True

# Generate/load utterance data and offensive classifier
clf_file = 'zoo:bot_adversarial_dialogue/multi_turn/model'
threshold = 0.5

save_file = f'./test_cases.responses.all.jsonl'
if not os.path.exists(save_file):
  EvalModel.main(task='test_cases', model_file="zoo:blender/blender_3B/model", skip_generation=False, batchsize=8, world_logs=save_file)

with open(save_file) as f:
  responses = [json.loads(line) for line in f]

humans = [response['dialog'][0][0]['text'] for response in responses]
bots = [response['dialog'][0][1]['text'] for response in responses]
texts = ['\n'.join([turn['text'] for turn in response['dialog'][0]]) for response in responses]

# Evaluate offensiveness
clf = OffensiveLanguageClassifier(custom_model_file=clf_file)
preds, confs = zip(*[clf.contains_offensive_language(text) for text in tqdm(texts)])
probs = np.array([conf if pred else (1 - conf) for pred, conf in zip(preds, confs)])
del clf  

# Save results to file
probs_save_file = save_file.rsplit('.jsonl', 1)[0] + f'.clf_file-{clf_file.replace("/", "-")}.probs.npy'
np.save(probs_save_file, probs)

toxic_frac = (probs >= threshold).mean()
print(f'{100. * toxic_frac}% Offensive')
print(100. * (probs >= threshold).mean(), f'% of Outputs Offensive')

offensive_idxs = list(np.where(probs >= threshold)[0])

offensive_zs_samples = []

for idx in offensive_idxs:
    offensive_zs_samples.append(texts[idx])
    
offensive_zs_samples_dict = {'Lyric': offensive_zs_samples}
df = pd.DataFrame.from_dict(offensive_zs_sample_dict)

In [ ]:
#Create a very small test set to compare generated text with the reality
test_set = df.sample(frac = 0.1)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [ ]:
#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [ ]:
test_set.head()

,level_0,index,SName,Lyric,Artist,Genre,True_end_lyrics
0,2946,3317,Do the Clam,(Words & music by Wayne - Weisman - Fuller). H...,Elvis Presley,Rock,Grab your barefoot baby by the hand. Turn and ...
1,12130,13349,Elevation,"High, higher than the sun. You shoot me from a...",U2,Rock,in the sky. You make me feel like I can fly. S...
2,596,640,Professional Torturer,Infatuation. Court well meant. 'Cause I'm the ...,Alanis Morissette,Rock,I renounce my name. Professional torturer. I d...
3,3733,4116,I Am Yours,I am yours. However distant you may be. There ...,Eric Clapton,Rock,me. Each memory that has left its trace with m...
4,11961,13175,Bombs Away,The general scratches his belly and thinks. Hi...,The Police,Rock,hard and sweet. A military man would love to m...


### Prepare the dataset

In [ ]:
class SongLyrics(Dataset):
    
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
                
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [ ]:
dataset = SongLyrics(df['Lyric'], truncate=True, gpt2_type="gpt2")

### Prepare training

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=1, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pth"),
            )
    return model

### Actual Training

In [ ]:
#Train the model on the specific data we have
model = train(dataset, model, tokenizer)

0it [00:00, ?it/s]

Training epoch 0
0


12400it [32:46,  6.31it/s]


Training epoch 1


0it [00:00, ?it/s]

...

12400it [32:46,  6.31it/s]


Training epoch 3


0it [00:00, ?it/s]

tensor(1.3163, device='cuda:0', grad_fn=<NllLossBackward>)


12400it [32:42,  6.32it/s]


In [ ]:
#Save the model to a pkl or something so it can be reused later on
torch.save(model, 'model.p')

### Text generation

In [ ]:
#Load the model to use it
model = torch.load('model.pt')

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [ ]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

In [ ]:
generated_lyrics = text_generation(test_set)

In [ ]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations

In [ ]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,index,SName,Lyric,Artist,Genre,True_end_lyrics,Generated_lyrics
0,2834,Don't Bring Me Down,"Hm hm hm. Hm hm hm. I'm on my own, nowhere to ...",David Bowie,Rock,me down. Until then I'll settle down. Say I'll...,me down. I want to see your face. I want to s...
1,6725,Flight,I've lost my balance. I fell from the trapeze....,Lifehouse,Rock,more falling). (No more striving). Only flying...,more falling). No more striving. No more hear...
2,7233,Black And Gold,If the fish swam out of the ocean. and grew le...,Lulu Santos,Rock,matter. 'cause if you're not really here. then...,magic words. 'cause if you're not real here. ...
3,2525,Hear My Train A Comin,"""YEAH, I SEE WE GOT A FEW FRIENDS LAYIN' ROUND...",Cássia Eller,Rock,YOU VERY MUCH THANK YOU VERY MUCH. THANX A LOT...,"S TO YOU. YES, YES, YES. YES. YES, YES."
4,1778,The Hills Of Mexico,'Twas in the town of griffin. In the year of s...,Bob Dylan,Rock,the cattle run. I tread towards the hiding pla...,"a horrid start. Struck my wagon, but couldn't..."


In [ ]:
test_set['Generated_lyrics'][7]

" in that. Yes we've heard the great thing. I know what you've heard. You told me we've been promised so much."

In [ ]:
test_set['True_end_lyrics'][7]

"the. Woman without pride x 5. You don't see things like I do. You don't see things. Like I do."

### Analyze performance

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6848624352005677

In [ ]:
#Rouge score
from rouge import Rouge
rouge=Rouge()

rouge.get_scores(test_set['Generated_lyrics'], test_set['True_end_lyrics'], avg=True)

{'rouge-1': {'f': 0.33620873608456614,
  'p': 0.3805105543072668,
  'r': 0.33900000000000013},
 'rouge-2': {'f': 0.24573902727265526,
  'p': 0.280178576490597,
  'r': 0.252700228832952},
 'rouge-l': {'f': 0.3756182538370741,
  'p': 0.40754447860807824,
  'r': 0.39803790370276443}}

# GPT2 without any fine Tuning

In [ ]:
import transformers
import torch

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
## Making a function that will generate text for us ##
def gen_text(prompt_text, tokenizer, model, n_seqs=1, max_length=374):
  # n_seqs is the number of sequences to generate
  # max_length is the maximum length of the sequence
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  # We are encoding the text using the gpt tokenizer. The return tensors are of type "pt"
  # since we are using PyTorch, not tensorflow
  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=max_length+len(encoded_prompt), # The model has to generate something, 
      # so we add the length of the original sequence to max_length
      temperature=1.0,
      top_k=0,
      top_p=0.9,
      repetition_penalty=1.2, # To ensure that we dont get repeated phrases
      do_sample=True,
      num_return_sequences=n_seqs
  ) # We feed the encoded input into the model.
  ## Getting the output ##
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # the _ indicates that the operation will be done in-place
  generated_sequences = []
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence)
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True, )) :]
    )
    generated_sequences.append(total_sequence)
  return generated_sequences

In [ ]:
#Generate sequences
gen_text(df['Lyric'][0],tokenizer,model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I feel so unsure. As I take your hand and lead to the dance floor. As the music dies, something in your eyes. Calls to mind the silver screen. And all its sad good-byes. I\'m never gonna dance again. Guilty feet have got no rhythm. Though it\'s easy to pretend. I know you are not a fool. Should\'ve known better than to cheat a friend. And waste the chance that I\'ve been given. So I\'m never gonna dance again. The way I danced with you. Time can never mend. The careless whispers of a good friend. To the heart and mind. Ignorance is kind. There\'s no comfort in the truth. Pain is all you\'ll find. I\'m never gonna dance again. Guilty feet have got no rhythm. Though it\'s easy to pretend. I know you are not a fool. Should\'ve known better than to cheat a friend. And waste this chance that I\'ve been given. So I\'m never gonna dance again. The way I danced with you. Never without your love. Tonight the music seems so loud. I wish that we could lose this crowd. Maybe it\'s better this wa

In [ ]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = gen_text(test_data['Lyric'][i], tokenizer, model)
    generated_lyrics.append(x)
  return generated_lyrics

generated_lyrics = text_generation(test_set)

In [ ]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations

In [ ]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,level_0,index,SName,Lyric,Artist,Genre,True_end_lyrics,Generated_lyrics
0,2946,3317,Do the Clam,(Words & music by Wayne - Weisman - Fuller). H...,Elvis Presley,Rock,Grab your barefoot baby by the hand. Turn and ...,
1,12130,13349,Elevation,"High, higher than the sun. You shoot me from a...",U2,Rock,in the sky. You make me feel like I can fly. S...,on earth.\nI start reading monographs about J...
2,596,640,Professional Torturer,Infatuation. Court well meant. 'Cause I'm the ...,Alanis Morissette,Rock,I renounce my name. Professional torturer. I d...,
3,3733,4116,I Am Yours,I am yours. However distant you may be. There ...,Eric Clapton,Rock,me. Each memory that has left its trace with m...,
4,11961,13175,Bombs Away,The general scratches his belly and thinks. Hi...,The Police,Rock,hard and sweet. A military man would love to m...,straight red hair.


In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4075527115657135

In [ ]:
!pip install rouge

In [ ]:
#Rouge score
from rouge import Rouge
rouge=Rouge()

rouge.get_scores(test_set['Generated_lyrics'], test_set['True_end_lyrics'], avg=True, ignore_empty=True)